## Importation des modules

In [ ]:
import cv2
import re
import numpy as np
import markdown
from html.parser import HTMLParser
import skvideo
skvideo.setFFmpegPath(r'C:\ProgramData\chocolatey\lib\ffmpeg\tools\ffmpeg\bin')
import skvideo.io
import skvideo.datasets
import skvideo.utils
import subprocess    
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup as Parse
from fractions import Fraction
from matplotlib import colors
import random

## Chargement de la vidéo

In [ ]:
IN_PATH = "./videos/snowman.mp4"
OUT_PATH = "./videos/snowman_track.mp4"

# Read video metadata
metadata = skvideo.io.ffprobe(IN_PATH)
#print(metadata)

fps = int(Fraction(metadata['video']['@avg_frame_rate']))
width = int(metadata['video']['@width'])
height = int(metadata['video']['@height'])
length = int(float(metadata['video']['@duration_ts']))

videoiter = skvideo.io.vreader(IN_PATH)

print(f"fps : {fps}    Width: {width}    Height: {height}    Duration: {length}")

## Lecture du fichier description

In [ ]:
with open("snowman.md","r",encoding="utf-8") as file:
    text = file.read()

html = markdown.markdown(text)
#display(HTML(html))

In [ ]:
def struct(data):
    regex = re.search('^(\d+:\d+:\d+)\s*([^/].*){0,1}(?:/(.*)){0,1}', data)
    timecode = regex.group(1)
    text = regex.group(2)
    color = regex.group(3)
    if not color:
        color = colors.to_rgb(random.choice(COLORS))
    else:
        color = colors.to_rgb(color)
    color = [int(col*255) for col in color]
    M, s, m = timecode.split(':')
    framecount = int(int(M)*60*fps + int(s)*fps + int(m)*fps/10)
    return dict(start=framecount, text = text, color=color)

def add_duration(data):
    new = []
    for i, cue in enumerate(data, 1):
        if i < len(data):
            cue['duration'] = data[i]['start'] - cue['start']
        else:
            cue['duration'] = length - cue['start']

        new.append(cue)
    return new

In [ ]:
WHITE = (255,255,255)
PURPLE = (255,255,0)
BLACK = (0,0,0)
GREEN = (0,255,0)
# https://matplotlib.org/stable/_images/sphx_glr_named_colors_003.png
COLORS = list(colors.cnames.keys())
font = cv2.FONT_HERSHEY_DUPLEX
stream_duration = fps*2

In [ ]:
parser = Parse(html)

streamers = [struct(stream.text) for stream in parser.find_all('li')]

sections = [struct(section.text) for section in parser.find_all('h1')]
sections = add_duration(sections)

subsections = [struct(sub_section.text) for sub_section in parser.find_all('h2')]
subsections = add_duration(subsections)

print(streamers)

In [ ]:
def imap(value, fromMin, fromMax, toMin, toMax):
    # Figure out how 'wide' each range is
    fromSpan = fromMax - fromMin
    toSpan = toMax - toMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - fromMin) / float(fromSpan)

    # Convert the 0-1 range into a value in the right range.
    return int(toMin + (valueScaled * toSpan))

def frames_to_TC (frames, fps):
    """
    # Hours: Divide frames by 86400 (# of frames in an hour at 24fps). Round down to nearest integer.
    # Minutes: Divide frames by 1440 (# of frames in a minute). This gives you the total number of minutes, which might be 122 for
    #          content that is 2 hours, 2 minutes, but you don't want the hours here. You're only interested in the extra 2 minutes.
    #          Modulo 60 will remove all full hours and return only the remaining minutes.
    # Seconds: frames % 1440 removes all full minutes and returns the number of remaining frames. 
    #          Divide that by 24 to convert to seconds, and round down to nearest integer.
    # Frames:  frames % 1440 removes all full minutes and returns the number of remaining frames. 
    #          Take that number and modulo 24 to removes all full seconds, leaving you with the remaining # of frames.
    # Lastly, take those variables and put them into a string with colons between each one.
    """
    #h = int(frames / 86400) 
    m = int(frames / (60*fps)) % 60 
    s = int((frames % (60*fps))/fps) 
    f = int(frames % (60*fps) % fps)
    mil = imap(f, 0,25,0,10)
    return f"{m:02}:{s:02}:{mil}"
    #return ( "%02d:%02d:%02d:%02d" % ( h, m, s, f))
    
def rect(im, frompos, size, color, weight):
    (fromx, fromy) = frompos
    (width, height) = size
    sub_img = im[fromy:fromy+height, fromx:fromx+width]
    col_rect = np.full(sub_img.shape, color, np.uint8)

    res = cv2.addWeighted(sub_img, weight, col_rect, 1, 0)

    # Putting the image back to its position
    im[fromy:fromy+height, fromx:fromx+width] = res
    return im

In [ ]:
writer = skvideo.io.FFmpegWriter(OUT_PATH)
videoiter = skvideo.io.vreader(IN_PATH)

sec = sections.copy()
sub = subsections.copy()

current_section = sec.pop(0)
current_subsection = sub.pop(0)

shift = 0
# Read until video is completed
for f, frame in enumerate(videoiter,0):
    if(frame.max() ==1):
        shift+=1
        continue
        
    f = f-shift

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    #print(f, sec[0]['start'], sec[0]['duration'])
    if f > sec[0]['start']:
        current_section = sec.pop(0)
        
    if f > sub[0]['start']:
        current_subsection = sub.pop(0)
     
    # STREAMER DRAWING
    # get all streamers less than 2 seconds from current frame
    next_streamers = [streamer for streamer in streamers if streamer['start'] > f > (streamer['start']-stream_duration)]
    for streamer in next_streamers:
        startx = int((stream_duration+f - streamer['start'])/stream_duration*width)
        frame = rect(frame, (startx,0), (50, height), streamer['color'], 0)

    # SECTION DRAWING
    frame = rect(frame, (0,0), (width, 50), BLACK, 0.5)
    frame = cv2.putText(frame, current_section['text'], (5, 30),font, 1, (255,255,255), 2)
    frame = cv2.rectangle(frame, (0,45), (int((f-current_section['start'])/current_section['duration']*width),50), GREEN, -1)
    
    # SUBSECTION DRAWING
    if current_subsection:
        rec_w = 100
        rec_h = 50
        frame = rect(frame, (width-rec_w,55), (width,rec_h), BLACK, 0.5)
        frame = cv2.putText(frame, current_subsection['text'], (width-rec_w+10, 55+rec_h-20),font, 1, (255,255,255), 2)
        if current_subsection['duration']:
            cue_start = current_subsection['start']
            subsection_percent = int((f-cue_start)/current_subsection['duration']*rec_w+width-rec_w)
            frame = cv2.rectangle(frame, (width-rec_w,rec_h+50), (subsection_percent,rec_h+55), PURPLE, -1)
    
    # TIMECODE DRAWING
    frame = rect(frame, (0,height-25), (80,height), BLACK, 0.8)
    frame = cv2.putText(frame, frames_to_TC(f, fps), (10,height-8),font, 0.5, (255,255,255), 1)

    # Display the resulting frame
    cv2.imshow('frame', frame)
    cv2.waitKey(25)
    if cv2.getWindowProperty('frame', cv2.WND_PROP_VISIBLE) <1: # Press Q on keyboard to  exit
        break
        

    writer.writeFrame(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
# Closes all the frames
cv2.destroyAllWindows()
writer.close()
subprocess.run(f"ffmpeg -i {OUT_PATH} -r 30 -i {IN_PATH} -r 30 -c copy ./videos/snowman_track_audio.mp4 -r 30")